In [6]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

# For my Mac
# os.chdir('//Users//patricklee//Desktop//파이썬//DACON//2020DACON_CUP')

# For my Desktop
os.chdir('C://Users//BIS_COM//data//dacon//2020')

In [3]:
df = pd.read_csv(r'./new_train_df.csv', parse_dates=['c_time'])

In [4]:
df

,c_time,id,ds_level,country_code,Total visit,browser,platform,Total User,사용자,세션,신규방문자,페이지뷰
0,2018-09-09 01:00:00,1.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,20.0,19.0,9.0,259.0
1,2018-09-09 03:00:00,1.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,10.0,10.0,2.0,102.0
2,2018-09-09 13:00:00,3.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,20.0,16.0,7.0,131.0
3,2018-09-09 18:00:00,1.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,13.0,12.0,2.0,47.0
4,2018-09-09 21:00:00,2.0,1.0,1.0,-99.0,-99.0,-99.0,-99.0,17.0,17.0,7.0,107.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13454,2019-12-30 07:00:00,-99.0,-99.0,-99.0,1.0,1.0,1.0,1.0,12.0,14.0,3.0,72.0
13455,2019-12-30 08:00:00,-99.0,-99.0,-99.0,7.0,3.0,3.0,6.0,29.0,29.0,5.0,274.0
13456,2019-12-30 12:00:00,-99.0,-99.0,-99.0,16.0,5.0,4.0,16.0,58.0,59.0,18.0,624.0
13457,2019-12-30 15:00:00,-99.0,-99.0,-99.0,36.0,7.0,5.0,34.0,100.0,100.0,28.0,1196.0


# 예제 첫 번째

In [8]:
epochs = 200
batch_size = 128
verbose = 1
nb_classes = 4
n_hidden = 128
validation_split = 0.2

rate = 0.2
rate_size = int(len(df) * rate)

train = df[:-rate_size]
test = df[-rate_size:]

In [9]:
x_col = train.columns[1:8]
y_col = train.columns[8:]
print(x_col, y_col)

Index(['id', 'ds_level', 'country_code', 'Total visit', 'browser', 'platform',
       'Total User'],
      dtype='object') Index(['사용자', '세션', '신규방문자', '페이지뷰'], dtype='object')


In [10]:
X_train = train[x_col]
Y_train = train[y_col]
X_test = test[x_col]
Y_test = test[y_col]

In [13]:
print(X_train.shape, X_test.shape)

(10768, 7) (2691, 7)


In [18]:
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(nb_classes, input_shape=(7,)
                            ))

In [19]:
model.compile(optimizer = 'SGD',
             loss = 'MSE',
             metrics=['accuracy'])

In [21]:
model.fit(X_train,Y_train,
         batch_size=batch_size, epochs=epochs,
         verbose=verbose, validation_split=validation_split)

Epoch 1/200
68/68 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.1438 - val_loss: nan - val_accuracy: 0.0065
Epoch 2/200
68/68 [==============================] - 0s 983us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 3/200
68/68 [==============================] - 0s 991us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 4/200
68/68 [==============================] - 0s 953us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 5/200
68/68 [==============================] - 0s 983us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 6/200
68/68 [==============================] - 0s 976us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 7/200
68/68 [==============================] - 0s 968us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 8/200
68/68 [========================

68/68 [==============================] - 0s 572us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 61/200
68/68 [==============================] - 0s 587us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 62/200
68/68 [==============================] - 0s 572us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 63/200
68/68 [==============================] - 0s 572us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 64/200
68/68 [==============================] - 0s 572us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 65/200
68/68 [==============================] - 0s 557us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 66/200
68/68 [==============================] - 0s 572us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 67/200
68/68 [===========================

68/68 [==============================] - 0s 587us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 120/200
68/68 [==============================] - 0s 587us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 121/200
68/68 [==============================] - 0s 572us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 122/200
68/68 [==============================] - 0s 572us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 123/200
68/68 [==============================] - 0s 572us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 124/200
68/68 [==============================] - 0s 587us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 125/200
68/68 [==============================] - 0s 587us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 126/200
68/68 [====================

68/68 [==============================] - 0s 616us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 179/200
68/68 [==============================] - 0s 601us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 180/200
68/68 [==============================] - 0s 631us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 181/200
68/68 [==============================] - 0s 601us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 182/200
68/68 [==============================] - 0s 601us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 183/200
68/68 [==============================] - 0s 587us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 184/200
68/68 [==============================] - 0s 580us/step - loss: nan - accuracy: 0.0072 - val_loss: nan - val_accuracy: 0.0065
Epoch 185/200
68/68 [====================

In [23]:
test_loss, test_acc = model.evaluate(X_test,Y_test)
print('Test ACC : ', test_acc)

85/85 [==============================] - 0s 317us/step - loss: nan - accuracy: 0.0000e+00
Test ACC :  0.0


# 두 번째 예제

In [28]:
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(n_hidden, input_shape=(7,),
                            activation='relu'))
model.add(keras.layers.Dense(n_hidden,
                            activation='relu'))
model.add(keras.layers.Dense(nb_classes,
                            activation='relu'))

In [32]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               1024      
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 516       
Total params: 18,052
Trainable params: 18,052
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer = 'SGD',
             loss = 'MSE',
             metrics=['accuracy'])

In [30]:
model.fit(X_train,Y_train,
         batch_size=batch_size, epochs=epochs,
         verbose=verbose, validation_split=validation_split)

Epoch 1/200
68/68 [==============================] - 0s 2ms/step - loss: 14984252416.0000 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 2/200
68/68 [==============================] - 0s 763us/step - loss: 1752734.3750 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 3/200
68/68 [==============================] - 0s 748us/step - loss: 1752734.2500 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 4/200
68/68 [==============================] - 0s 760us/step - loss: 1752734.3750 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 5/200
68/68 [==============================] - 0s 781us/step - loss: 1752734.3750 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 6/200
68/68 [==============================] - 0s 734us/step - loss: 1752734.6250 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 7/200
68/68 [==============================] - 0s 735us/step - loss: 1752734.500

68/68 [==============================] - 0s 777us/step - loss: 1752734.3750 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 55/200
68/68 [==============================] - 0s 748us/step - loss: 1752734.6250 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 56/200
68/68 [==============================] - 0s 748us/step - loss: 1752734.7500 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 57/200
68/68 [==============================] - 0s 747us/step - loss: 1752734.2500 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 58/200
68/68 [==============================] - 0s 763us/step - loss: 1752734.6250 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 59/200
68/68 [==============================] - 0s 748us/step - loss: 1752734.7500 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 60/200
68/68 [==============================] - 0s 748us/step - loss: 1752734.0000 - accu

68/68 [==============================] - 0s 748us/step - loss: 1752734.7500 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 108/200
68/68 [==============================] - 0s 763us/step - loss: 1752734.5000 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 109/200
68/68 [==============================] - 0s 763us/step - loss: 1752734.6250 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 110/200
68/68 [==============================] - 0s 763us/step - loss: 1752734.3750 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 111/200
68/68 [==============================] - 0s 748us/step - loss: 1752734.6250 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 112/200
68/68 [==============================] - 0s 748us/step - loss: 1752734.6250 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 113/200
68/68 [==============================] - 0s 750us/step - loss: 1752734.3750 

68/68 [==============================] - 0s 755us/step - loss: 1752734.6250 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 161/200
68/68 [==============================] - 0s 733us/step - loss: 1752734.7500 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 162/200
68/68 [==============================] - 0s 748us/step - loss: 1752734.6250 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 163/200
68/68 [==============================] - 0s 749us/step - loss: 1752734.6250 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 164/200
68/68 [==============================] - 0s 735us/step - loss: 1752734.6250 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 165/200
68/68 [==============================] - 0s 735us/step - loss: 1752734.2500 - accuracy: 0.0072 - val_loss: 7409.7153 - val_accuracy: 0.0065
Epoch 166/200
68/68 [==============================] - 0s 750us/step - loss: 1752734.5000 

In [31]:
test_loss, test_acc = model.evaluate(X_test,Y_test)
print('Test ACC : ', test_acc)

85/85 [==============================] - 0s 352us/step - loss: 28818.3340 - accuracy: 0.0000e+00
Test ACC :  0.0


# 세 번째 예제

In [33]:
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(n_hidden, input_shape=(7,),
                            activation='relu'))
model.add(keras.layers.Dense(n_hidden,
                            activation='relu'))
model.add(keras.layers.Dense(nb_classes,
                            activation='relu'))

In [34]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               1024      
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 516       
Total params: 18,052
Trainable params: 18,052
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(optimizer = 'adam',
             loss = 'MSE',
             metrics=['accuracy'])

In [36]:
model.fit(X_train,Y_train,
         batch_size=batch_size, epochs=epochs,
         verbose=verbose, validation_split=validation_split)

Epoch 1/200
68/68 [==============================] - 0s 5ms/step - loss: 1293443.3750 - accuracy: 0.8968 - val_loss: 4523.7695 - val_accuracy: 0.9935
Epoch 2/200
68/68 [==============================] - 0s 821us/step - loss: 565494.8750 - accuracy: 0.9928 - val_loss: 4736.7231 - val_accuracy: 0.9935
Epoch 3/200
68/68 [==============================] - 0s 836us/step - loss: 549740.5625 - accuracy: 0.9928 - val_loss: 4472.1035 - val_accuracy: 0.9935
Epoch 4/200
68/68 [==============================] - 0s 803us/step - loss: 545801.3750 - accuracy: 0.9928 - val_loss: 4287.3472 - val_accuracy: 0.9935
Epoch 5/200
68/68 [==============================] - 0s 836us/step - loss: 541947.2500 - accuracy: 0.9928 - val_loss: 3733.9385 - val_accuracy: 0.9935
Epoch 6/200
68/68 [==============================] - 0s 822us/step - loss: 538230.8750 - accuracy: 0.9928 - val_loss: 3545.2256 - val_accuracy: 0.9935
Epoch 7/200
68/68 [==============================] - 0s 793us/step - loss: 532010.7500 - accura

Epoch 55/200
68/68 [==============================] - 0s 792us/step - loss: 356253.7188 - accuracy: 0.9928 - val_loss: 3444.8765 - val_accuracy: 0.9935
Epoch 56/200
68/68 [==============================] - 0s 807us/step - loss: 354959.9688 - accuracy: 0.9928 - val_loss: 3188.7100 - val_accuracy: 0.9935
Epoch 57/200
68/68 [==============================] - 0s 835us/step - loss: 355177.2188 - accuracy: 0.9928 - val_loss: 3357.7085 - val_accuracy: 0.9935
Epoch 58/200
68/68 [==============================] - 0s 821us/step - loss: 356404.9688 - accuracy: 0.9928 - val_loss: 3128.1160 - val_accuracy: 0.9935
Epoch 59/200
68/68 [==============================] - 0s 836us/step - loss: 355047.3125 - accuracy: 0.9928 - val_loss: 3221.4050 - val_accuracy: 0.9935
Epoch 60/200
68/68 [==============================] - 0s 821us/step - loss: 360544.8750 - accuracy: 0.9928 - val_loss: 3482.8464 - val_accuracy: 0.9935
Epoch 61/200
68/68 [==============================] - 0s 865us/step - loss: 355172.0000 

68/68 [==============================] - 0s 778us/step - loss: 354231.2188 - accuracy: 0.9927 - val_loss: 3108.7090 - val_accuracy: 0.9935
Epoch 109/200
68/68 [==============================] - 0s 807us/step - loss: 354730.7500 - accuracy: 0.9928 - val_loss: 3358.2476 - val_accuracy: 0.9935
Epoch 110/200
68/68 [==============================] - 0s 795us/step - loss: 355516.3438 - accuracy: 0.9928 - val_loss: 3584.7905 - val_accuracy: 0.9935
Epoch 111/200
68/68 [==============================] - 0s 807us/step - loss: 352850.7500 - accuracy: 0.9927 - val_loss: 3377.3689 - val_accuracy: 0.9935
Epoch 112/200
68/68 [==============================] - 0s 803us/step - loss: 354227.0312 - accuracy: 0.9927 - val_loss: 3212.4199 - val_accuracy: 0.9935
Epoch 113/200
68/68 [==============================] - 0s 825us/step - loss: 353338.8125 - accuracy: 0.9928 - val_loss: 3406.8420 - val_accuracy: 0.9935
Epoch 114/200
68/68 [==============================] - 0s 802us/step - loss: 354716.2812 - accur

68/68 [==============================] - 0s 836us/step - loss: 352885.3438 - accuracy: 0.9927 - val_loss: 3223.4968 - val_accuracy: 0.9935
Epoch 162/200
68/68 [==============================] - 0s 821us/step - loss: 352027.9688 - accuracy: 0.9928 - val_loss: 3641.0212 - val_accuracy: 0.9935
Epoch 163/200
68/68 [==============================] - 0s 821us/step - loss: 353379.7812 - accuracy: 0.9927 - val_loss: 3555.7512 - val_accuracy: 0.9935
Epoch 164/200
68/68 [==============================] - 0s 807us/step - loss: 354484.1250 - accuracy: 0.9928 - val_loss: 3310.0623 - val_accuracy: 0.9935
Epoch 165/200
68/68 [==============================] - 0s 807us/step - loss: 354198.2500 - accuracy: 0.9927 - val_loss: 3800.4832 - val_accuracy: 0.9935
Epoch 166/200
68/68 [==============================] - 0s 813us/step - loss: 353567.5938 - accuracy: 0.9928 - val_loss: 3142.6460 - val_accuracy: 0.9935
Epoch 167/200
68/68 [==============================] - 0s 809us/step - loss: 355010.2812 - accur

In [37]:
test_loss, test_acc = model.evaluate(X_test,Y_test)
print('Test ACC : ', test_acc)

85/85 [==============================] - 0s 352us/step - loss: 13468.5752 - accuracy: 1.0000
Test ACC :  1.0


In [38]:
model.compile(optimizer = 'RMSProp',
             loss = 'MSE',
             metrics=['accuracy'])

In [39]:
model.fit(X_train,Y_train,
         batch_size=batch_size, epochs=epochs,
         verbose=verbose, validation_split=validation_split)

Epoch 1/200
68/68 [==============================] - 0s 2ms/step - loss: 355380.2812 - accuracy: 0.9927 - val_loss: 3806.3420 - val_accuracy: 0.9935
Epoch 2/200
68/68 [==============================] - 0s 792us/step - loss: 354878.3125 - accuracy: 0.9927 - val_loss: 3825.0281 - val_accuracy: 0.9935
Epoch 3/200
68/68 [==============================] - 0s 792us/step - loss: 355049.6250 - accuracy: 0.9927 - val_loss: 3675.4656 - val_accuracy: 0.9935
Epoch 4/200
68/68 [==============================] - 0s 792us/step - loss: 354103.2812 - accuracy: 0.9927 - val_loss: 3055.6042 - val_accuracy: 0.9935
Epoch 5/200
68/68 [==============================] - 0s 777us/step - loss: 354216.3438 - accuracy: 0.9928 - val_loss: 3894.3372 - val_accuracy: 0.9935
Epoch 6/200
68/68 [==============================] - 0s 763us/step - loss: 353955.7500 - accuracy: 0.9927 - val_loss: 3977.2200 - val_accuracy: 0.9935
Epoch 7/200
68/68 [==============================] - 0s 777us/step - loss: 354584.3125 - accurac

Epoch 55/200
68/68 [==============================] - 0s 792us/step - loss: 354050.2500 - accuracy: 0.9928 - val_loss: 3208.2458 - val_accuracy: 0.9935
Epoch 56/200
68/68 [==============================] - 0s 821us/step - loss: 354264.1250 - accuracy: 0.9928 - val_loss: 3538.0977 - val_accuracy: 0.9935
Epoch 57/200
68/68 [==============================] - 0s 865us/step - loss: 354106.9688 - accuracy: 0.9927 - val_loss: 3583.0874 - val_accuracy: 0.9935
Epoch 58/200
68/68 [==============================] - 0s 807us/step - loss: 354242.0000 - accuracy: 0.9928 - val_loss: 3469.8796 - val_accuracy: 0.9935
Epoch 59/200
68/68 [==============================] - 0s 777us/step - loss: 354659.6562 - accuracy: 0.9928 - val_loss: 3148.9675 - val_accuracy: 0.9935
Epoch 60/200
68/68 [==============================] - 0s 768us/step - loss: 353967.0000 - accuracy: 0.9927 - val_loss: 3877.1196 - val_accuracy: 0.9935
Epoch 61/200
68/68 [==============================] - 0s 777us/step - loss: 353793.8750 

Epoch 109/200
68/68 [==============================] - 0s 792us/step - loss: 353061.6562 - accuracy: 0.9928 - val_loss: 3772.5474 - val_accuracy: 0.9935
Epoch 110/200
68/68 [==============================] - 0s 763us/step - loss: 353683.4375 - accuracy: 0.9928 - val_loss: 3405.5222 - val_accuracy: 0.9935
Epoch 111/200
68/68 [==============================] - 0s 766us/step - loss: 353145.9062 - accuracy: 0.9928 - val_loss: 4203.0190 - val_accuracy: 0.9935
Epoch 112/200
68/68 [==============================] - 0s 829us/step - loss: 353396.0938 - accuracy: 0.9927 - val_loss: 3737.3879 - val_accuracy: 0.9935
Epoch 113/200
68/68 [==============================] - 0s 909us/step - loss: 353120.1875 - accuracy: 0.9926 - val_loss: 4111.6426 - val_accuracy: 0.9935
Epoch 114/200
68/68 [==============================] - 0s 851us/step - loss: 352941.0625 - accuracy: 0.9927 - val_loss: 3873.3816 - val_accuracy: 0.9935
Epoch 115/200
68/68 [==============================] - 0s 798us/step - loss: 35363

68/68 [==============================] - 0s 779us/step - loss: 353446.9062 - accuracy: 0.9926 - val_loss: 2939.2598 - val_accuracy: 0.9935
Epoch 163/200
68/68 [==============================] - 0s 777us/step - loss: 353029.0000 - accuracy: 0.9928 - val_loss: 3172.7227 - val_accuracy: 0.9935
Epoch 164/200
68/68 [==============================] - 0s 822us/step - loss: 352562.3750 - accuracy: 0.9927 - val_loss: 3572.4504 - val_accuracy: 0.9935
Epoch 165/200
68/68 [==============================] - 0s 833us/step - loss: 354050.7188 - accuracy: 0.9928 - val_loss: 3492.6848 - val_accuracy: 0.9935
Epoch 166/200
68/68 [==============================] - 0s 821us/step - loss: 353482.7188 - accuracy: 0.9928 - val_loss: 3180.9297 - val_accuracy: 0.9935
Epoch 167/200
68/68 [==============================] - 0s 798us/step - loss: 353527.8125 - accuracy: 0.9928 - val_loss: 3451.8884 - val_accuracy: 0.9935
Epoch 168/200
68/68 [==============================] - 0s 807us/step - loss: 353482.4375 - accur

In [40]:
test_loss, test_acc = model.evaluate(X_test,Y_test)
print('Test ACC : ', test_acc)

85/85 [==============================] - 0s 344us/step - loss: 12195.5234 - accuracy: 1.0000
Test ACC :  1.0
